In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import glob
os.getcwd()

# Note email cpm updated based on the Q4 client number
# Flipp hosted not included in wide version

'/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1'

In [2]:
new_quarter="2020Q1"
new_quarter_start_date="2020-02-02"
new_quarter_end_date="2020-05-02"
os.getcwd()

'/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1'

In [3]:
TMR_up_to_2019Q4=pd.read_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2019Q4/output/BL_MMM_long_cumu_up_to_2019Q4_JL_2020-12-16.csv",dtype=str)
TMR_up_to_2019Q4['impression']=TMR_up_to_2019Q4['impression'].fillna(0).astype(float).astype(int)
TMR_up_to_2019Q4['click']=TMR_up_to_2019Q4['click'].fillna(0).astype(float).astype(int)
TMR_up_to_2019Q4['cost']=TMR_up_to_2019Q4['cost'].fillna(0).astype(float)


In [4]:
TMR_up_to_2019Q4_no_DMA=TMR_up_to_2019Q4.groupby(["week date","media","submedia","placement"])['impression','click','cost'].sum().reset_index()
print(TMR_up_to_2019Q4_no_DMA['week date'].min())
print(TMR_up_to_2019Q4_no_DMA['week date'].max())
# TMR_up_to_2019Q4_no_DMA

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


2016-09-25
2020-01-26


In [5]:
TMR_up_to_2019Q4_no_DMA.head(2)

,week date,media,submedia,placement,impression,click,cost
0,2016-09-25,Email,xx,xx,11581550,113834,5278.691191
1,2016-10-02,Digital,Flipp,Flipp App,74691,37222,10794.380000


In [6]:
def get_week_end_date_BL(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

# TV

In [7]:
# Due to no impression by week, 2 out of the 4 tabs are not included
# Sinclair(Local) & DISH ignored
(160000+125000)/(3373431+125000+160000+84504)

0.07614345426784061

In [8]:
# Email: Big Lots Q120 TMR from Jessica, Fhursday, Aug 27, 2020 at 9:09 PM
'''
Media   (To confirm if net or gross, last quarter was net)                                        
National $3,373,431
DISH $125,000
Sinclair $160,000
DRTV $84,504

Unfortunately, DISH was unable to break their spots out for the TMR. I have attached the MSA we were provided. There were a total of 773 spots for $125,000 and 5,748,000 impressions.
'''
national_tv_cost=3373431
sinclair_tv_cost=160000
drtv_tv_cost=84504
# DISH ignored
TV_TMR_data=pd.DataFrame()

In [9]:
TV_print_excel=pd.ExcelFile("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1/input_2020Q1/Big Lots Q1 TMR 8.26.20.xlsx")
print(TV_print_excel.sheet_names)
                      
df_TV_national=TV_print_excel.parse("National",dtype=str,skiprows=2)
df_TV_sinclair=TV_print_excel.parse("Sinclair",dtype=str,skiprows=2)
df_TV_DRTV=TV_print_excel.parse("DRTV",dtype=str,skiprows=2)

print(df_TV_national.shape,df_TV_sinclair.shape,df_TV_DRTV.shape)


['National', 'Sinclair', 'DRTV', 'DISH', 'Circ Spend']
(3477, 18) (148, 18) (186, 18)


In [10]:
df_TV_national=df_TV_national[["Q1",'National',"WeTV",' 2/10/2018',3935]]
df_TV_national.columns=["Q1 2020",'National',"Network","Air Date",' Imp(000) - Women 25-54']
# df_TV_national['Air Date']=df_TV_national['Air Date'].apply(lambda x: x.strip())
df_TV_national=df_TV_national[pd.notnull(df_TV_national['Air Date'])]

df_TV_national=df_TV_national.rename(columns={"Network":"placement"})
df_TV_national['date']=df_TV_national['Air Date'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%Y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
df_TV_national['impression']=df_TV_national[' Imp(000) - Women 25-54'].astype(float)

df_TV_national['media']="TV"
df_TV_national['submedia']="National"
df_TV_national['click']=0
df_TV_national['week date']=df_TV_national['date'].apply(get_week_end_date_BL)
df_TV_national['week date']=df_TV_national['week date'].apply(lambda x: x-datetime.timedelta(days=6))

sum_national_cable_impr=df_TV_national['impression'].sum()
df_TV_national['cost']=df_TV_national['impression']/sum_national_cable_impr*national_tv_cost
print("df_TV_national['cost'].sum()",df_TV_national['cost'].sum())
df_TV_national=df_TV_national[TMR_up_to_2019Q4_no_DMA.columns.tolist()]


df_TV_national['cost'].sum() 3373431.0


In [11]:
df_TV_sinclair.head(2)

,Q1,Post,Unnamed: 2,Baltimore,WeTV,Spot Cable,2018-02-05 00:00:00,W25-54,2/10/2018,Saturday,5:27 PM,30,LIVE PD,21.25,0.77,3935,"6740 COMCAST, Baltimore Interconnect-Cable",Unnamed: 17
0,Q1 2020 Sinclair,Post,Sinclair,"Asheville, NC",WLOS,NaN,NaN,W25-54,2020-05-01 03:38:08,Friday,1970-01-01 08:38:08,00:30,NaN,NaN,NaN,NaN,LGTS1336000H,NaN
1,Q1 2020 Sinclair,Post,Sinclair,Mobile / Pensacola,WPMI,NaN,NaN,W25-55,2020-04-30 00:41:55,Thursday,1970-01-01 05:41:55,00:30,NaN,NaN,NaN,NaN,LGTS1336000H,NaN


In [12]:
'''
# To confirm if impression available esle ignore

df_TV_sinclair=df_TV_sinclair[["Q1",'Unnamed: 2',"WeTV",' 2/10/2018',3935]]
df_TV_sinclair.columns=["Q1 2020",'National',"Network","Air Date",' Imp(000) - Women 25-54']
# df_TV_sinclair['Air Date']=df_TV_sinclair['Air Date'].apply(lambda x: x.strip())
df_TV_sinclair=df_TV_sinclair[pd.notnull(df_TV_sinclair['Air Date'])]

df_TV_sinclair=df_TV_sinclair.rename(columns={"Network":"placement"})
df_TV_sinclair['date']=df_TV_sinclair['Air Date'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%Y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
df_TV_sinclair['impression']=df_TV_sinclair[' Imp(000) - Women 25-54'].astype(float)*1000

df_TV_sinclair['media']="TV"
df_TV_sinclair['submedia']="National"
df_TV_sinclair['click']=0
df_TV_sinclair['week date']=df_TV_sinclair['date'].apply(get_week_end_date_BL)
df_TV_sinclair['week date']=df_TV_sinclair['week date'].apply(lambda x: x-datetime.timedelta(days=6))

sum_national_cable_impr=df_TV_sinclair['impression'].sum()
df_TV_sinclair['cost']=df_TV_sinclair['impression']/sum_national_cable_impr*national_tv_cost
print("df_TV_sinclair['cost'].sum()",df_TV_sinclair['cost'].sum())
df_TV_sinclair=df_TV_sinclair[TMR_up_to_2019Q4_no_DMA.columns.tolist()]
'''

'\n# To confirm if impression available esle ignore\n\ndf_TV_sinclair=df_TV_sinclair[["Q1",\'Unnamed: 2\',"WeTV",\' 2/10/2018\',3935]]\ndf_TV_sinclair.columns=["Q1 2020",\'National\',"Network","Air Date",\' Imp(000) - Women 25-54\']\n# df_TV_sinclair[\'Air Date\']=df_TV_sinclair[\'Air Date\'].apply(lambda x: x.strip())\ndf_TV_sinclair=df_TV_sinclair[pd.notnull(df_TV_sinclair[\'Air Date\'])]\n\ndf_TV_sinclair=df_TV_sinclair.rename(columns={"Network":"placement"})\ndf_TV_sinclair[\'date\']=df_TV_sinclair[\'Air Date\'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%Y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())\ndf_TV_sinclair[\'impression\']=df_TV_sinclair[\' Imp(000) - Women 25-54\'].astype(float)*1000\n\ndf_TV_sinclair[\'media\']="TV"\ndf_TV_sinclair[\'submedia\']="National"\ndf_TV_sinclair[\'click\']=0\ndf_TV_sinclair[\'week date\']=df_TV_sinclair[\'date\'].apply(get_week_end_date_BL)\ndf_TV_sinclair[\'week date\']=df_TV_sinclair[\'week da

In [13]:
df_TV_DRTV.columns.tolist()

['Q1',
 'Post',
 'Nielsen Keeping Trac',
 'Baltimore',
 'WeTV',
 'Spot Cable',
 datetime.datetime(2018, 2, 5, 0, 0),
 'W25-54',
 ' 2/10/2018',
 'Saturday',
 ' 5:27 PM',
 30,
 'LIVE PD',
 21.25,
 0.77,
 3935,
 '6740 COMCAST, Baltimore Interconnect-Cable',
 'Unnamed: 17']

In [14]:
df_TV_DRTV=df_TV_DRTV[["Q1",'Baltimore',"WeTV",' 2/10/2018',3935]]
df_TV_DRTV.columns=["Q1 2020",'National',"Network","Air Date",' Imp(000) - Women 25-54']
# df_TV_DRTV['Air Date']=df_TV_DRTV['Air Date'].apply(lambda x: x.strip())
df_TV_DRTV=df_TV_DRTV[pd.notnull(df_TV_DRTV['Air Date'])]

df_TV_DRTV=df_TV_DRTV.rename(columns={"Network":"placement"})
df_TV_DRTV['date']=df_TV_DRTV['Air Date'].apply(lambda x: datetime.datetime.strptime(x," %m/%d/%Y").date() if "/" in x else datetime.datetime.strptime(x,"%Y-%m-%d %H:%M:%S").date())
df_TV_DRTV['impression']=df_TV_DRTV[' Imp(000) - Women 25-54'].astype(float)

df_TV_DRTV['media']="TV"
df_TV_DRTV['submedia']="National"
df_TV_DRTV['click']=0
df_TV_DRTV['week date']=df_TV_DRTV['date'].apply(get_week_end_date_BL)
df_TV_DRTV['week date']=df_TV_DRTV['week date'].apply(lambda x: x-datetime.timedelta(days=6))

sum_DRTV_impr=df_TV_DRTV['impression'].sum()
df_TV_DRTV['cost']=df_TV_DRTV['impression']/sum_DRTV_impr*drtv_tv_cost
print("df_TV_DRTV['cost'].sum()",df_TV_DRTV['cost'].sum())
df_TV_DRTV=df_TV_DRTV[TMR_up_to_2019Q4_no_DMA.columns.tolist()]


df_TV_DRTV['cost'].sum() 84504.0


In [15]:
TV_TMR_data=df_TV_national.append(df_TV_DRTV)
TV_TMR_data['week date']=TV_TMR_data['week date'].astype(str)
print("TV_TMR_data['cost'].sum()",TV_TMR_data['cost'].sum())
print("TV_TMR_data['impression'].sum()",TV_TMR_data['impression'].sum())


TV_TMR_data['cost'].sum() 3457935.0
TV_TMR_data['impression'].sum() 131940595.0


In [16]:
TV_TMR_data.shape

(3663, 7)

In [17]:
TV_TMR_data=TV_TMR_data.groupby(['week date','media','submedia','placement'])['impression','click','cost'].sum().reset_index()
TV_TMR_data.shape

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


(57, 7)

In [18]:
TV_TMR_data=TV_TMR_data[TV_TMR_data['week date']>=new_quarter_start_date]
TV_TMR_data=TV_TMR_data[TV_TMR_data['week date']<=new_quarter_end_date]
print(TV_TMR_data.shape)
print(TV_TMR_data['week date'].nunique())
print(TV_TMR_data['week date'].min())
print(TV_TMR_data['week date'].max())


df_TV_summary=TV_TMR_data.groupby(['submedia'])['impression','cost'].sum().reset_index()
df_TV_summary['impression']=df_TV_summary['impression'].astype(int)
df_TV_summary

(57, 7)
8
2020-02-02
2020-03-29


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  if __name__ == '__main__':


,submedia,impression,cost
0,National,131940595,3457935.0


In [19]:
df_qc_tv_lastyear=TMR_up_to_2019Q4_no_DMA[TMR_up_to_2019Q4_no_DMA['week date']>="2019-02-03"]
df_qc_tv_lastyear=df_qc_tv_lastyear[df_qc_tv_lastyear['week date']<="2019-05-04"]
df_qc_tv_lastyear=df_qc_tv_lastyear[df_qc_tv_lastyear['media']=="TV"]
print(df_qc_tv_lastyear['cost'].sum(),df_qc_tv_lastyear['impression'].sum())

# In line

2661712.1000000006 235130070


In [20]:
df_TV_summary['cost'].sum(),df_TV_summary['impression'].sum()

(3457935.0000000005, 131940595)

In [21]:
3457935.0000000005/2661712.1000000006

1.299139377245195

In [22]:
131940595/235130070

0.5611387560935953

In [23]:
print(datetime.datetime.now())

2020-12-16 21:51:59.255619


# Print Circulation

In [24]:
week_1=datetime.date(2020,2,2)
df_week_2020Q1=pd.DataFrame({"week date":week_1},index=range(13))
df_week_2020Q1['Fiscal Week']=range(13)
df_week_2020Q1['week date']=df_week_2020Q1['Fiscal Week'].apply(lambda x: week_1+datetime.timedelta(days=x*7))
df_week_2020Q1['Fiscal Week']=df_week_2020Q1['Fiscal Week']+1

In [25]:
df_print_circ=TV_print_excel.parse("Circ Spend",dtype=str,skiprows=None)
df_print_circ['Fiscal Week']=df_print_circ['Fiscal Week'].apply(lambda x: int(x.split("(")[0]))
df_print_circ=pd.merge(df_print_circ,df_week_2020Q1,on="Fiscal Week")
df_print_circ=df_print_circ.rename(columns={"Impressions":"impression","Investment":"cost"})
df_print_circ['click']=0
df_print_circ['cost']=df_print_circ['cost'].astype(float)
df_print_circ['media']="Circulation"
df_print_circ['submedia']="xx"
df_print_circ['placement']="xx"

df_print_circ=df_print_circ[TMR_up_to_2019Q4_no_DMA.columns.tolist()]
print(df_print_circ.shape)
print(df_print_circ['week date'].nunique(),df_print_circ['week date'].min(),df_print_circ['week date'].max())
print(pd.to_datetime(df_print_circ['week date']).dt.day_name().unique())

# Update for this quarter some circulation was printed but not delivered
df_print_circ['cost']=np.where(df_print_circ['impression']==0,0,df_print_circ['cost'])

(13, 7)
13 2020-02-02 2020-04-26
['Sunday']


# Others from Datorama

In [26]:
list_datorama_files=glob.glob("./input_2020Q1/datorama_files/*.csv")
list_datorama_files.sort()
list_datorama_files

['./input_2020Q1/datorama_files/Digital Weekly Media Data  (Search).csv',
 './input_2020Q1/datorama_files/Digital Weekly Media Data without Placement (excluded FB).csv',
 './input_2020Q1/datorama_files/E-circular_Flipp & Print Circular & Email by Week.csv',
 './input_2020Q1/datorama_files/Social Daily Media Data.csv']

In [27]:
df_1_search=pd.read_csv(list_datorama_files[0])
df_1_search.columns=[x.lower() for x in df_1_search.columns.tolist()]
df_1_search=df_1_search.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_1_search=df_1_search[['week','media','submedia','placement','impression','click','cost']]
df_1_search['submedia']=df_1_search['submedia'].replace({"Paid SEM Campaign":"SEM"}).replace({"Shopping Campaign":"PLA"})
df_1_search=df_1_search.sort_values(['media','submedia','placement','week'])
df_1_search.head(2)

,week,media,submedia,placement,impression,click,cost
0,2020-02-02 00:00,Digital,PLA,Bing,22817.0,532.0,1157.74
7,2020-02-09 00:00,Digital,PLA,Bing,54035.0,1184.0,2376.13


In [28]:
df_2_digital=pd.read_csv(list_datorama_files[1])
df_2_digital.columns=[x.lower() for x in df_2_digital.columns.tolist()]
df_2_digital=df_2_digital.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_2_digital=df_2_digital[['week','media','submedia','impression','click','cost','partner','tactic']]

df_2_digital_YT=df_2_digital[df_2_digital['partner']=="YouTube"]
df_2_digital_YT['placement']=df_2_digital_YT['submedia']
df_2_digital_YT['submedia']="YouTube"
df_2_digital_YT=df_2_digital_YT[df_1_search.columns.tolist()]

df_2_digital_Pandora=df_2_digital[df_2_digital['partner']=="Pandora"]
df_2_digital_Pandora['placement']=df_2_digital_Pandora['submedia']
df_2_digital_Pandora['submedia']="Pandora"
df_2_digital_Pandora=df_2_digital_Pandora[df_1_search.columns.tolist()]

df_2_digital_Programmatic=df_2_digital[~df_2_digital['partner'].isin(["YouTube",'Pandora'])]
df_2_digital_Programmatic['placement']=df_2_digital_Programmatic['submedia']
df_2_digital_Programmatic['submedia']="Programmatic"
df_2_digital_Programmatic=df_2_digital_Programmatic[df_1_search.columns.tolist()]

df_2_digital=df_2_digital_YT.append(df_2_digital_Pandora).append(df_2_digital_Programmatic)


df_2_digital.head(2)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.p

,week,media,submedia,placement,impression,click,cost
5,2020-04-26 00:00,Digital,YouTube,Multiplatform Video,0.0,0.0,81146.498788
7,2020-04-26 00:00,Digital,YouTube,Programmatic Video,14513759.0,14234.0,0.000000


In [29]:
df_3_email_ecirc=pd.read_csv(list_datorama_files[2])
df_3_email_ecirc.columns=[x.lower() for x in df_3_email_ecirc.columns.tolist()]
df_3_email_ecirc=df_3_email_ecirc.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})
df_3_email_ecirc=df_3_email_ecirc[['week','media','placement','impression','click','cost']]
df_3_email_ecirc.insert(2,"submedia",np.nan)
df_3_email_ecirc['submedia']=np.where(df_3_email_ecirc['media']=="Digital","Flipp","xx")
df_3_email_ecirc['placement']=np.where(df_3_email_ecirc['media']=="Digital",df_3_email_ecirc['placement'],"xx")

df_3_email_ecirc.head(3)

,week,media,submedia,placement,impression,click,cost
0,2020-04-26 00:00,Email,xx,xx,197929448.0,2309286.0,90213.178160
1,2020-04-26 00:00,Digital,Flipp,Hosted,231812.0,201120.0,0.000000
2,2020-04-19 00:00,Email,xx,xx,165367801.0,1770414.0,75372.083559


In [30]:
df_4_social=pd.read_csv(list_datorama_files[3])
df_4_social.columns=[x.lower() for x in df_4_social.columns.tolist()]
df_4_social=df_4_social.rename(columns={"impressions":"impression","clicks":"click","media cost":"cost"})

df_4_social=df_4_social[['week','media','submedia','partner','impression','click','cost']]
df_4_social=df_4_social.rename(columns={"partner":"placement"})
df_4_social.head(3)

,week,media,submedia,placement,impression,click,cost
0,2020-02-02 00:00,Digital,Social,Facebook,8431154.0,24568.0,55517.76
1,2020-02-02 00:00,Digital,Social,Pinterest,535341.0,2094.0,2795.37
2,2020-02-02 00:00,Digital,Social,Pinterest,13544.0,22.0,0.00


In [31]:
df_all_digital_from_datorama=pd.concat([df_1_search,df_2_digital,df_3_email_ecirc,df_4_social])
df_all_digital_from_datorama['week']=df_all_digital_from_datorama['week'].apply(lambda x: x.split(" ")[0])
df_all_digital_from_datorama=df_all_digital_from_datorama.rename(columns={"week":"week date"})

# QC week date as 
df_all_digital_from_datorama['week_as_date'] = pd.to_datetime(df_all_digital_from_datorama['week date'])
df_all_digital_from_datorama['day_of_week'] = df_all_digital_from_datorama['week_as_date'].dt.day_name()
print(df_all_digital_from_datorama['day_of_week'].unique())
del df_all_digital_from_datorama['week_as_date']
del df_all_digital_from_datorama['day_of_week']

df_all_digital_from_datorama.head(2)

['Sunday']


,week date,media,submedia,placement,impression,click,cost
0,2020-02-02,Digital,PLA,Bing,22817.0,532.0,1157.74
7,2020-02-09,Digital,PLA,Bing,54035.0,1184.0,2376.13


In [32]:
print(df_all_digital_from_datorama['week date'].nunique())
print(df_all_digital_from_datorama['week date'].min())
print(df_all_digital_from_datorama['week date'].max())

13
2020-02-02
2020-04-26


# Cumulative

In [33]:
Q1_2020_only_TMR=df_all_digital_from_datorama.append(df_print_circ).append(TV_TMR_data)
Q1_2020_only_TMR['week date']=Q1_2020_only_TMR['week date'].astype(str)
print(Q1_2020_only_TMR['week date'].nunique())
print(Q1_2020_only_TMR['week date'].min())
print(Q1_2020_only_TMR['week date'].max())


13
2020-02-02
2020-04-26


In [34]:
# Update email cost based on the confirmation from the client

# Email:  BL QC of non-Media Storm media for 2019 Q2 from Bryan Kipp, 2020-12-15 5:29 PM
Q1_2020_only_TMR_email=Q1_2020_only_TMR[Q1_2020_only_TMR['media']=="Email"]

cost_q1_total=522624
cost_q1_impression=Q1_2020_only_TMR_email['impression'].sum()
print(Q1_2020_only_TMR_email.shape)
Q1_2020_only_TMR_email['cost']=Q1_2020_only_TMR_email['impression']*cost_q1_total/cost_q1_impression

print(Q1_2020_only_TMR_email['impression'].sum(),Q1_2020_only_TMR_email['click'].sum(),Q1_2020_only_TMR_email['cost'].sum())


(13, 7)
2132375888.0 18609176.0 522623.9999999999


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [35]:
Q1_2020_only_TMR_others=Q1_2020_only_TMR[Q1_2020_only_TMR['media']!="Email"]
Q1_2020_only_TMR=Q1_2020_only_TMR_others.append(Q1_2020_only_TMR_email)
#
Cum_Q1_2020_TMR=TMR_up_to_2019Q4.append(Q1_2020_only_TMR)

In [36]:
try:
    os.stat("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1/output/")
except:
    os.mkdir("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1/output/")

Q1_2020_only_TMR.to_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1/output/BL_MMM_2020Q1_Only_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
Cum_Q1_2020_TMR.to_csv("/mnt/clients/juba/hqjubaapp02/sharefolder/Media/TMR/TMR_data/Up_to_2020Q1/output/BL_MMM_long_cumu_up_to_2020Q1_JL_"+str(datetime.datetime.now().date())+".csv",index=False)
print(datetime.datetime.now())

2020-12-16 21:52:14.939420
